In [1]:
import os
opj = os.path.join

import ccxt 
import matplotlib.pyplot as plt 
import pandas as pd 
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor 
from sklearn.metrics import f1_score, mean_squared_error, mean_absolute_error
import torch

In [2]:
df_files = [
    "crypto-gt.csv",
    "deberta.csv",
    "default.csv",
    "roberta.csv",
    "bert.csv"
]

experiment_name = "sentiment_no_neutral_news_4hours"
saveas = opj("results", f"{experiment_name}.csv")
checkpoint_save_dir = opj("ckpts", experiment_name)
exp_times = 10
results = {}

In [3]:
def experiment(df_file, N):
    exp_name = df_file.split(".")[0]
    # load data
    chart_df = pd.read_csv(df_file)
    i1 = 0
    i2 = len(chart_df)-1
    st = "2018-02-14 14:00:00"
    en = "2022-04-15 23:00:00"
    filt = []
    for index, row in chart_df.iterrows():
        if chart_df.iloc[i1]['datetime'] < row.datetime and row.datetime < st:
            i1 = index
        if en < row.datetime and row.datetime < chart_df.iloc[i2]['datetime']:
            i2 = index
    chart_df = chart_df[i1+1:i2]
    chart_df.index = range(len(chart_df))
    
    # Train High Model
    ## train/validation/test split 
    train_size = int(chart_df.shape[0] * 0.8) 
    train_df = chart_df.iloc[:train_size,:] 

    val_size = int(chart_df.shape[0] * 0.1) 
    val_df = chart_df.iloc[train_size:train_size+val_size,:]  

    test_df = chart_df.iloc[train_size+val_size:, :] 
    
    categorical_columns = ["months", "days", "hours"]
    features = train_df.columns

    cat_idxs = [0, 1, 2] 
    cat_dims = [13, 32, 25] 

    tabnet_params = {"cat_idxs":cat_idxs, 
                     "cat_dims":cat_dims, 
                     "cat_emb_dim":1, 
                     "optimizer_fn":torch.optim.Adam,
                     "seed": N
                    } 
    
    input_columns = [] 
    skips = ['low_delta', 'years', 'datetime', 'sent_2']
    for col in train_df.columns:
        if col in skips: continue
        input_columns.append(col) 

    X_train = train_df[input_columns].values 
    Y_train = train_df['high_delta'].values 
    Y_train = Y_train.reshape((-1,1))

    X_val = val_df[input_columns].values
    Y_val = val_df['high_delta'].values 
    Y_val = Y_val.reshape((-1,1))

    X_test = test_df[input_columns].values 
    Y_test = test_df['high_delta'].values  
    Y_test = Y_test.reshape((-1,1))
    
    reg_high = TabNetRegressor(**tabnet_params) 

    reg_high.fit(X_train, Y_train, 
                 eval_set=[(X_val, Y_val)], 
                 max_epochs=100, 
                 patience=100)  
    
    Y_pred = reg_high.predict(X_test).flatten() 
    
    exp = f"{exp_name}-high"
    if exp not in results:
        results[exp] = []
        
    results[exp].append(
        {
            "MSE": mean_squared_error(Y_test, Y_pred),
            "MAE": mean_absolute_error(Y_test, Y_pred)
        }
    )
    
    saveas = opj(checkpoint_save_dir, f"{exp}-{N}")
    reg_high.save_model(saveas)
    
    # Train Low Model
    input_columns = []             
    skips = ['high_delta', 'years', 'datetime', 'sent_2']
    for col in train_df.columns:
        if col in skips: continue
        input_columns.append(col) 

    X_train = train_df[input_columns].values 
    Y_train = train_df['low_delta'].values 
    Y_train = Y_train.reshape((-1,1))

    X_val = val_df[input_columns].values
    Y_val = val_df['low_delta'].values 
    Y_val = Y_val.reshape((-1,1))

    X_test = test_df[input_columns].values 
    Y_test = test_df['low_delta'].values  
    Y_test = Y_test.reshape((-1,1))
    
    reg_low = TabNetRegressor(**tabnet_params) 

    reg_low.fit(X_train, Y_train, 
                eval_set=[(X_val, Y_val)], 
                max_epochs=200, 
                patience=200)  
    
    Y_pred = reg_low.predict(X_test).flatten() 
    exp = f"{exp_name}-low"
    if exp not in results:
        results[exp] = []
        
    results[exp].append(
        {
            "MSE": mean_squared_error(Y_test, Y_pred),
            "MAE": mean_absolute_error(Y_test, Y_pred)
        }
    )
    
    saveas = opj(checkpoint_save_dir, f"{exp}-{N}")
    reg_high.save_model(saveas)

In [ ]:
for df_file in df_files:
    for i in range(exp_times):
        experiment(df_file, i)

Device used : cuda
epoch 0  | loss: 1.17783 | val_0_mse: 0.02346 |  0:00:00s
epoch 1  | loss: 0.15851 | val_0_mse: 0.00708 |  0:00:01s
epoch 2  | loss: 0.03701 | val_0_mse: 0.00468 |  0:00:02s
epoch 3  | loss: 0.01516 | val_0_mse: 0.00608 |  0:00:02s
epoch 4  | loss: 0.00629 | val_0_mse: 0.00186 |  0:00:03s
epoch 5  | loss: 0.00424 | val_0_mse: 0.0007  |  0:00:04s
epoch 6  | loss: 0.00266 | val_0_mse: 0.00021 |  0:00:05s
epoch 7  | loss: 0.00199 | val_0_mse: 0.00034 |  0:00:05s
epoch 8  | loss: 0.00164 | val_0_mse: 0.00023 |  0:00:06s
epoch 9  | loss: 0.00155 | val_0_mse: 0.00031 |  0:00:07s
epoch 10 | loss: 0.00128 | val_0_mse: 0.00024 |  0:00:08s
epoch 11 | loss: 0.00121 | val_0_mse: 0.00024 |  0:00:09s
epoch 12 | loss: 0.00125 | val_0_mse: 0.00023 |  0:00:10s
epoch 13 | loss: 0.0011  | val_0_mse: 0.00023 |  0:00:11s
epoch 14 | loss: 0.00109 | val_0_mse: 0.0002  |  0:00:12s
epoch 15 | loss: 0.00105 | val_0_mse: 0.00021 |  0:00:13s
epoch 16 | loss: 0.00105 | val_0_mse: 0.00019 |  0:00

epoch 37 | loss: 0.00093 | val_0_mse: 0.00014 |  0:00:29s
epoch 38 | loss: 0.0009  | val_0_mse: 0.00017 |  0:00:30s
epoch 39 | loss: 0.00092 | val_0_mse: 0.00014 |  0:00:31s
epoch 40 | loss: 0.00088 | val_0_mse: 0.00014 |  0:00:31s
epoch 41 | loss: 0.00085 | val_0_mse: 0.00014 |  0:00:32s
epoch 42 | loss: 0.00086 | val_0_mse: 0.00014 |  0:00:33s
epoch 43 | loss: 0.00087 | val_0_mse: 0.00015 |  0:00:34s
epoch 44 | loss: 0.00085 | val_0_mse: 0.00014 |  0:00:34s
epoch 45 | loss: 0.00087 | val_0_mse: 0.00014 |  0:00:35s
epoch 46 | loss: 0.00087 | val_0_mse: 0.00013 |  0:00:36s
epoch 47 | loss: 0.00087 | val_0_mse: 0.00014 |  0:00:37s
epoch 48 | loss: 0.00082 | val_0_mse: 0.00016 |  0:00:37s
epoch 49 | loss: 0.00082 | val_0_mse: 0.00013 |  0:00:38s
epoch 50 | loss: 0.0008  | val_0_mse: 0.00014 |  0:00:39s
epoch 51 | loss: 0.0008  | val_0_mse: 0.00014 |  0:00:40s
epoch 52 | loss: 0.0008  | val_0_mse: 0.00016 |  0:00:41s
epoch 53 | loss: 0.0008  | val_0_mse: 0.00014 |  0:00:42s
epoch 54 | los

epoch 179| loss: 0.00038 | val_0_mse: 0.00013 |  0:02:21s
epoch 180| loss: 0.00035 | val_0_mse: 0.00012 |  0:02:22s
epoch 181| loss: 0.00036 | val_0_mse: 0.0001  |  0:02:22s
epoch 182| loss: 0.00036 | val_0_mse: 0.00011 |  0:02:23s
epoch 183| loss: 0.00036 | val_0_mse: 0.00011 |  0:02:24s
epoch 184| loss: 0.00034 | val_0_mse: 0.00011 |  0:02:25s
epoch 185| loss: 0.00035 | val_0_mse: 0.00012 |  0:02:26s
epoch 186| loss: 0.00034 | val_0_mse: 0.00013 |  0:02:26s
epoch 187| loss: 0.00034 | val_0_mse: 0.00015 |  0:02:27s
epoch 188| loss: 0.00033 | val_0_mse: 0.00014 |  0:02:28s
epoch 189| loss: 0.00034 | val_0_mse: 0.00011 |  0:02:29s
epoch 190| loss: 0.00033 | val_0_mse: 0.0001  |  0:02:30s
epoch 191| loss: 0.00033 | val_0_mse: 0.0001  |  0:02:31s
epoch 192| loss: 0.00033 | val_0_mse: 0.0001  |  0:02:31s
epoch 193| loss: 0.00032 | val_0_mse: 0.0001  |  0:02:32s
epoch 194| loss: 0.00032 | val_0_mse: 0.00011 |  0:02:33s
epoch 195| loss: 0.00033 | val_0_mse: 0.00011 |  0:02:34s
epoch 196| los

epoch 12 | loss: 0.0011  | val_0_mse: 0.00018 |  0:00:10s
epoch 13 | loss: 0.00112 | val_0_mse: 0.00014 |  0:00:10s
epoch 14 | loss: 0.00108 | val_0_mse: 0.0002  |  0:00:11s
epoch 15 | loss: 0.00104 | val_0_mse: 0.00017 |  0:00:12s
epoch 16 | loss: 0.00105 | val_0_mse: 0.00022 |  0:00:13s
epoch 17 | loss: 0.00103 | val_0_mse: 0.0002  |  0:00:14s
epoch 18 | loss: 0.00101 | val_0_mse: 0.00021 |  0:00:14s
epoch 19 | loss: 0.00099 | val_0_mse: 0.00018 |  0:00:15s
epoch 20 | loss: 0.00097 | val_0_mse: 0.00018 |  0:00:16s
epoch 21 | loss: 0.00096 | val_0_mse: 0.00018 |  0:00:17s
epoch 22 | loss: 0.00095 | val_0_mse: 0.00017 |  0:00:17s
epoch 23 | loss: 0.00094 | val_0_mse: 0.00019 |  0:00:18s
epoch 24 | loss: 0.00095 | val_0_mse: 0.00018 |  0:00:19s
epoch 25 | loss: 0.00092 | val_0_mse: 0.00017 |  0:00:20s
epoch 26 | loss: 0.00092 | val_0_mse: 0.00017 |  0:00:21s
epoch 27 | loss: 0.00091 | val_0_mse: 0.00016 |  0:00:21s
epoch 28 | loss: 0.00092 | val_0_mse: 0.0002  |  0:00:22s
epoch 29 | los

epoch 154| loss: 0.00036 | val_0_mse: 0.00014 |  0:01:59s
epoch 155| loss: 0.00036 | val_0_mse: 0.00013 |  0:02:00s
epoch 156| loss: 0.00035 | val_0_mse: 0.00014 |  0:02:01s
epoch 157| loss: 0.00035 | val_0_mse: 0.00013 |  0:02:01s
epoch 158| loss: 0.00035 | val_0_mse: 0.00013 |  0:02:02s
epoch 159| loss: 0.00034 | val_0_mse: 0.00014 |  0:02:03s
epoch 160| loss: 0.00035 | val_0_mse: 0.00015 |  0:02:04s
epoch 161| loss: 0.00035 | val_0_mse: 0.00013 |  0:02:05s
epoch 162| loss: 0.00034 | val_0_mse: 0.00013 |  0:02:05s
epoch 163| loss: 0.00034 | val_0_mse: 0.00014 |  0:02:06s
epoch 164| loss: 0.00034 | val_0_mse: 0.00013 |  0:02:07s
epoch 165| loss: 0.00035 | val_0_mse: 0.00014 |  0:02:08s
epoch 166| loss: 0.00038 | val_0_mse: 0.00016 |  0:02:08s
epoch 167| loss: 0.00037 | val_0_mse: 0.00014 |  0:02:09s
epoch 168| loss: 0.00034 | val_0_mse: 0.00013 |  0:02:10s
epoch 169| loss: 0.00033 | val_0_mse: 0.00014 |  0:02:11s
epoch 170| loss: 0.00033 | val_0_mse: 0.00014 |  0:02:12s
epoch 171| los

...

epoch 53 | loss: 0.00067 | val_0_mse: 0.00015 |  0:00:46s
epoch 54 | loss: 0.00066 | val_0_mse: 0.00015 |  0:00:47s
epoch 55 | loss: 0.00066 | val_0_mse: 0.00015 |  0:00:47s
epoch 56 | loss: 0.00064 | val_0_mse: 0.00014 |  0:00:48s
epoch 57 | loss: 0.00064 | val_0_mse: 0.00015 |  0:00:49s
epoch 58 | loss: 0.00064 | val_0_mse: 0.00015 |  0:00:50s
epoch 59 | loss: 0.00063 | val_0_mse: 0.00015 |  0:00:51s
epoch 60 | loss: 0.00063 | val_0_mse: 0.00014 |  0:00:52s
epoch 61 | loss: 0.00063 | val_0_mse: 0.00016 |  0:00:53s
epoch 62 | loss: 0.00061 | val_0_mse: 0.00015 |  0:00:54s
epoch 63 | loss: 0.00062 | val_0_mse: 0.00016 |  0:00:55s
epoch 64 | loss: 0.00061 | val_0_mse: 0.00014 |  0:00:55s
epoch 65 | loss: 0.0006  | val_0_mse: 0.00014 |  0:00:56s
epoch 66 | loss: 0.00059 | val_0_mse: 0.00015 |  0:00:57s
epoch 67 | loss: 0.00059 | val_0_mse: 0.00014 |  0:00:58s
epoch 68 | loss: 0.00059 | val_0_mse: 0.00015 |  0:00:59s
epoch 69 | loss: 0.00057 | val_0_mse: 0.00014 |  0:01:00s
epoch 70 | los

epoch 195| loss: 0.0003  | val_0_mse: 9e-05   |  0:02:49s
epoch 196| loss: 0.00029 | val_0_mse: 9e-05   |  0:02:50s
epoch 197| loss: 0.00028 | val_0_mse: 9e-05   |  0:02:50s
epoch 198| loss: 0.0003  | val_0_mse: 9e-05   |  0:02:51s
epoch 199| loss: 0.00028 | val_0_mse: 9e-05   |  0:02:52s
Stop training because you reached max_epochs = 200 with best_epoch = 199 and best_val_0_mse = 9e-05
Best weights from best epoch are automatically used!
Successfully saved model at ckpts/sentiment_no_neutral_news_4hours/roberta-low-8.zip
Device used : cuda
epoch 0  | loss: 0.45427 | val_0_mse: 0.09178 |  0:00:00s
epoch 1  | loss: 0.10313 | val_0_mse: 0.03186 |  0:00:01s
epoch 2  | loss: 0.03581 | val_0_mse: 0.00127 |  0:00:02s
epoch 3  | loss: 0.01006 | val_0_mse: 0.00201 |  0:00:03s
epoch 4  | loss: 0.00481 | val_0_mse: 0.00107 |  0:00:04s
epoch 5  | loss: 0.00318 | val_0_mse: 0.00034 |  0:00:05s
epoch 6  | loss: 0.00259 | val_0_mse: 0.00015 |  0:00:06s
epoch 7  | loss: 0.00179 | val_0_mse: 0.0002  |

epoch 28 | loss: 0.00081 | val_0_mse: 0.00016 |  0:00:25s
epoch 29 | loss: 0.00081 | val_0_mse: 0.00016 |  0:00:25s
epoch 30 | loss: 0.00079 | val_0_mse: 0.00016 |  0:00:26s
epoch 31 | loss: 0.0008  | val_0_mse: 0.00017 |  0:00:27s
epoch 32 | loss: 0.00079 | val_0_mse: 0.00016 |  0:00:28s
epoch 33 | loss: 0.00078 | val_0_mse: 0.00015 |  0:00:29s
epoch 34 | loss: 0.00075 | val_0_mse: 0.00015 |  0:00:30s
epoch 35 | loss: 0.00074 | val_0_mse: 0.00015 |  0:00:31s
epoch 36 | loss: 0.00074 | val_0_mse: 0.00014 |  0:00:31s
epoch 37 | loss: 0.00073 | val_0_mse: 0.00014 |  0:00:32s
epoch 38 | loss: 0.00072 | val_0_mse: 0.00014 |  0:00:33s
epoch 39 | loss: 0.00071 | val_0_mse: 0.00014 |  0:00:34s
epoch 40 | loss: 0.00072 | val_0_mse: 0.00014 |  0:00:35s
epoch 41 | loss: 0.00071 | val_0_mse: 0.00014 |  0:00:36s
epoch 42 | loss: 0.00078 | val_0_mse: 0.00013 |  0:00:37s
epoch 43 | loss: 0.0007  | val_0_mse: 0.00014 |  0:00:38s
epoch 44 | loss: 0.00069 | val_0_mse: 0.00013 |  0:00:38s
epoch 45 | los

epoch 170| loss: 0.00013 | val_0_mse: 3e-05   |  0:02:28s
epoch 171| loss: 0.00013 | val_0_mse: 3e-05   |  0:02:29s
epoch 172| loss: 0.00013 | val_0_mse: 4e-05   |  0:02:30s
epoch 173| loss: 0.00015 | val_0_mse: 2e-05   |  0:02:31s
epoch 174| loss: 0.00013 | val_0_mse: 4e-05   |  0:02:31s
epoch 175| loss: 0.00016 | val_0_mse: 5e-05   |  0:02:32s
epoch 176| loss: 0.00015 | val_0_mse: 3e-05   |  0:02:33s
epoch 177| loss: 0.00013 | val_0_mse: 2e-05   |  0:02:34s
epoch 178| loss: 0.00012 | val_0_mse: 2e-05   |  0:02:34s
epoch 179| loss: 0.00012 | val_0_mse: 4e-05   |  0:02:35s
epoch 180| loss: 0.00012 | val_0_mse: 3e-05   |  0:02:36s
epoch 181| loss: 0.00013 | val_0_mse: 3e-05   |  0:02:37s
epoch 182| loss: 0.00011 | val_0_mse: 3e-05   |  0:02:38s
epoch 183| loss: 0.00012 | val_0_mse: 3e-05   |  0:02:38s
epoch 184| loss: 0.00015 | val_0_mse: 2e-05   |  0:02:40s
epoch 185| loss: 0.00016 | val_0_mse: 4e-05   |  0:02:40s
epoch 186| loss: 0.00013 | val_0_mse: 4e-05   |  0:02:41s
epoch 187| los

epoch 3  | loss: 0.02451 | val_0_mse: 0.00184 |  0:00:03s
epoch 4  | loss: 0.01249 | val_0_mse: 0.00134 |  0:00:04s
epoch 5  | loss: 0.00593 | val_0_mse: 0.00068 |  0:00:05s
epoch 6  | loss: 0.00486 | val_0_mse: 0.00045 |  0:00:06s
epoch 7  | loss: 0.00318 | val_0_mse: 0.00037 |  0:00:07s
epoch 8  | loss: 0.00247 | val_0_mse: 0.00031 |  0:00:08s
epoch 9  | loss: 0.00211 | val_0_mse: 0.00023 |  0:00:09s
epoch 10 | loss: 0.0018  | val_0_mse: 0.00019 |  0:00:10s
epoch 11 | loss: 0.00158 | val_0_mse: 0.00015 |  0:00:11s
epoch 12 | loss: 0.00154 | val_0_mse: 0.00016 |  0:00:12s
epoch 13 | loss: 0.00143 | val_0_mse: 0.00015 |  0:00:13s
epoch 14 | loss: 0.00142 | val_0_mse: 0.00015 |  0:00:14s
epoch 15 | loss: 0.00127 | val_0_mse: 0.00015 |  0:00:15s
epoch 16 | loss: 0.00123 | val_0_mse: 0.00016 |  0:00:16s
epoch 17 | loss: 0.00118 | val_0_mse: 0.00016 |  0:00:17s
epoch 18 | loss: 0.00118 | val_0_mse: 0.00016 |  0:00:18s
epoch 19 | loss: 0.00121 | val_0_mse: 0.00016 |  0:00:18s
epoch 20 | los

In [ ]:
if not os.path.exists(checkpoint_save_dir):
    os.makedirs(checkpoint_save_dir, exist_ok=True)
with open(saveas, 'w') as f:
    for key in results.keys():
        f.write("%s, %s\n" % (key, results[key]))